### Author : BlackViper(Harshit Saxena)
#### Created at 16:14:20 26-01-2018
______

In [10]:
# Loading packages
import os
import pandas as pd
import numpy as np
import datetime
from dateutil import parser
import dateutil
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import pylab
from scipy.sparse import csr_matrix
import os.path as op
import nimfa as nf

In [11]:
def importData(dataDir='/home/harshit/Desktop/Analytics/Dota'):
    
    Data = pd.read_csv(dataDir+"/preprocessedtrain.csv")
    testdata = pd.read_csv(dataDir+"/preprocessedtest.csv")
    Data['id'] = Data['id'].astype("category")            #converting into categorical variables
    Data['user_id'] = Data['user_id'].astype("category")
    Data['hero_id'] = Data['hero_id'].astype("category")
    testdata['id'] = testdata['id'].astype("category")            #converting into categorical variables
    testdata['user_id'] = testdata['user_id'].astype("category")
    testdata['hero_id'] = testdata['hero_id'].astype("category")
    Data = Data.dropna(how='any')
    return Data, testdata

In [12]:
data, testdata = importData()

In [13]:
data.shape

(29022, 40)

In [14]:
print('Splitting data.')    
train,test = train_test_split(data,test_size=0.2,random_state=1)

Splitting data.


In [15]:
np.unique(data.user_id)

array([   1,    2,    3, ..., 2990, 2991, 2992])

In [19]:
def RMSE(y_pred,y_true):
    return np.sqrt(np.mean((y_true - y_pred)**2))

def RMSE_matrix(prediction, test_mat):
    mask = test_mat>0; mask[mask==True]=1; mask[mask==False]=0
    prediction = np.multiply(prediction, mask)
    return (np.sum((prediction - test_mat)**2)/np.sum(mask))**0.5

In [16]:
def alspreprocess(ratingData, test, train):
    n_u = len(ratingData['user_id'].cat.categories)
    print n_u
    n_m = len(ratingData['hero_id'].cat.categories)+5
    print n_m
    test_col = np.array(test['user_id'].values)-1
    print test_col.max()
    test_row = np.array(test['hero_id'].values)-1
    print test_row.max()
    test_dat = np.array(test['kda_ratio'].values)
    train_col = np.array(train['user_id'].values)-1
    print train_col.max()
    train_row = np.array(train['hero_id'].values)-1
    print train_row.max()
    train_dat = np.array(train['kda_ratio'].values)
    train_mat = (csr_matrix((train_dat, (train_row, train_col)), shape=(n_m, n_u)).toarray()).T
    test_mat = (csr_matrix((test_dat, (test_row, test_col)), shape=(n_m, n_u)).toarray()).T
    return test_mat, train_mat

In [17]:
test_mat, train_mat = alspreprocess(data, test, train)

2992
120
2991
119
2991
119


In [ ]:
als  = nf.Lsnmf(train_mat,seed="random_vcol",rank=25,max_iter = 50, beta = 0.5)
als_fit = als.factorize()
        
U = np.array(als_fit.basis())
V = np.array(als_fit.coef())
predictedMat = np.round(np.array(np.dot(U, V)),2)
trainRMSE = RMSE_matrix(predictedMat,train_mat)
testRMSE = RMSE_matrix(predictedMat,test_mat)
print "ALS\nTrain RMSE : %f  Test RMSE : %f"%(trainRMSE, testRMSE)

In [51]:
# def als(matrix, n_factors=8,n_iterations=15, lambda_=10):
# 	"""
#      Carries out ALS decomposition of a given matrix and returns
#      the predicted matrix and the decomposed matrices X and Y
     
#      INPUT
#      matrix : Numpy matrix to be decomposed
#      n_factors : Number of features for the decomposed matrices
#      lambda : Regularization factor
     
#      OUTPUT
#      prediction : ALS predicted matrix i.e product of X.T and Y
#      X,Y : Decomposed matrices computed by ALS
 
#      """
# 	m, n = matrix.shape
# 	Q = matrix
# 	W = Q > 0.5
# 	W = W.astype(int)
# 	print('X and Y randomly initialzied.')
# 	X = 5 * np.random.rand(m, n_factors) 
# 	Y = 5 * np.random.rand(n_factors, n)
# 	for ii in range(n_iterations):
# 		for u, Wu in enumerate(W):
# 			X[u] = np.linalg.solve(np.dot(Y, np.dot(np.diag(Wu), Y.T)) + lambda_ * np.eye(n_factors),
# 	                                np.dot(Y, np.dot(np.diag(Wu), Q[u].T))).T
# 		for i, Wi in enumerate(W.T):
# 			Y[:,i] = np.linalg.solve(np.dot(X.T, np.dot(np.diag(Wi), X)) + lambda_ * np.eye(n_factors),
# 	                                np.dot(X.T, np.dot(np.diag(Wi), Q[:, i])))
# 		print('{}th iteration is completed of {}'.format(ii + 1,n_iterations))
# 	prediction = np.dot(X,Y)
# 	print('Done.')
# 	return prediction, X, Y


In [22]:
data.columns

Index([u'id', u'hero_id', u'user_id', u'num_games', u'num_wins', u'kda_ratio',
       u'seqno', u'primary_attr_agi', u'primary_attr_int', u'primary_attr_str',
       u'attack_type_Melee', u'attack_type_Ranged', u'base_health',
       u'base_health_regen', u'base_mana', u'base_mana_regen', u'base_armor',
       u'base_magic_resistance', u'base_attack_min', u'base_attack_max',
       u'base_strength', u'base_agility', u'base_intelligence',
       u'strength_gain', u'agility_gain', u'intelligence_gain',
       u'attack_range', u'projectile_speed', u'attack_rate', u'move_speed',
       u'turn_rate', u'role_Carry', u'role_Nuker', u'role_Disabler',
       u'role_Escape', u'role_Pusher', u'role_Initiator', u'role_Jungler',
       u'role_Durable', u'role_Support'],
      dtype='object')

In [63]:
features = [ u'num_games', u'primary_attr_agi', u'primary_attr_int', u'primary_attr_str',
       u'attack_type_Melee', u'attack_type_Ranged', u'base_health',
       u'base_health_regen', u'base_mana', u'base_mana_regen', u'base_armor',
       u'base_magic_resistance', u'base_attack_min', u'base_attack_max',
       u'base_strength', u'base_agility', u'base_intelligence',
       u'strength_gain', u'agility_gain', u'intelligence_gain',
       u'attack_range', u'projectile_speed', u'attack_rate', u'move_speed',
       u'turn_rate', u'role_Carry', u'role_Nuker', u'role_Disabler',
       u'role_Escape', u'role_Pusher', u'role_Initiator', u'role_Jungler',
       u'role_Durable', u'role_Support']

In [64]:
def alspostprocess(data, prediction, features, user_features, movie_features, n_features=60):
    """
    Adds ALS values obtained from ALS decomposition of ratings matrix as
    feature to dataFrame.
    Also adds movie and user features obtained ALS decomposition to 
    corresponding samples
    INPUT     
    data : DataFrame to which ALS, user_features and movie_features 
           are to be added.
           
    prediction : Numpy array containing ALS prediction
    
    user_features : ALS user features
    
    movie_features : ALS movie features
    
    n_features : Number of user_features and movie_features to be added to the DataFrame
    
    OUTPUT
    data : DataFrame with ALS, user_features and movie_features added
    
    features : List of feature names to be used for further modelling
    """
    

    data['ALS'] = prediction[data.loc[:, 'user_id'].astype(int)-1, data.loc[:, 'hero_id'].astype(int)-1]
    features.append('ALS')
    
    total_features = len(movie_features)
    if n_features>total_features:
        n_features = total_features
        
    for i in range(n_features):
        data["UserFeature{}".format(i)] = user_features[data.loc[:, 'user_id'].astype(int)-1, i]
        features.append("UserFeature{}".format(i))
        data["MovieFeature{}".format(i)] = movie_features[i, data.loc[:, 'hero_id'].astype(int)-1]
        features.append("MovieFeature{}".format(i))
    return data, features

In [65]:
finaldata, feature = alspostprocess(data, predictedMat,features=features, user_features=U, movie_features=V)

In [76]:
finaltest, feature_test = alspostprocess(testdata, predictedMat,features=features, user_features=U, movie_features=V)

In [82]:
ydata = data.pop('kda_ratio')

In [66]:
train,test = train_test_split(finaldata,test_size=0.2,random_state=1)
ytrain = train.pop('kda_ratio')
ytest = test.pop('kda_ratio')

In [67]:
train.shape

(23217, 90)

In [75]:
print('Training Random Forest.')
model = RandomForestRegressor(250,max_depth=15, oob_score=True,random_state=42, n_jobs=-1)
model.fit(train[feature],ytrain)

trainRMSE = RMSE(ytrain,model.predict(train[feature]))
testRMSE = RMSE(ytest,model.predict(test[feature]))
print "Train RMSE : %f  Test RMSE : %f"%(trainRMSE, testRMSE)

Training Random Forest.
Train RMSE : 437.246827  Test RMSE : 729.503177


In [83]:
print('Training Random Forest.')
model = RandomForestRegressor(250,max_depth=15, oob_score=True,random_state=42, n_jobs=-1)
model.fit(data[feature],ydata)
print(' Predicting and submission.')
finaltest['predict_kda'] = model.predict(finaltest[feature])
submit = pd.DataFrame(data=finaltest.id, columns=['id'])
submit['kda_ratio'] = finaltest['predict_kda']
submit.to_csv('submit_rf_250_15.csv', index=False)

Training Random Forest.


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=-1,
           oob_score=True, random_state=42, verbose=0, warm_start=False)